In [1]:
# !pip install torchtext
# importing required libraries
import mediapipe as mp
# from mediapipe.framework.formats import landmark_pb2
# from mediapipe.tasks import python
# from mediapipe.tasks.python import vision
# from mediapipe import solutions
import os
# import pandas as pd
import numpy as np
import cv2
import torch.nn as nn
import torch
import torch.nn.functional as F
# import torch 
# from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import math
import warnings
import pandas as pd
import numpy as np
# import seaborn as sns
# import torchtext
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
print(torch.__version__)

2024-07-12 07:17:42.977021: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.3.1+cu121


In [2]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")

Using: cpu


In [42]:
training_set = datasets.ImageFolder('/home/Rupali/ISL-to-text/Word Level ISL/Frames_Word_Level')

In [49]:
dataloader = torch.utils.data.DataLoader(training_set, batch_size=1036, shuffle=True)

In [ ]:
def one_hot(indx,size):
    ret = [0]*size
    ret[indx] = 1
    return ret

In [72]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size):
    size = 1036
    i = 1
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for X, y in dataloader:
        # Compute prediction and loss
        X = np.array(X)
        y = torch.tensor([y])
        # y = torch.Tensor(torch.nn.functional.one_hot(torch.tensor(y), num_classes=114))
        print(y)
        pred = model(X)
        print(pred)
        loss = loss_fn(pred, one_hot(y,126))

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # loss, current = loss.item(), batch * batch_size + len(X)
        print(f"loss: {loss:>7f}  [{i:>5d}/{size:>5d}]")
        i+=1


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [30]:
import model_arc

model = model_arc.Transformer(126,1000,114,3)
model = model.to('cuda')
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.045)


In [73]:
for t in range(100):
    print(f"Epoch: {t+1}\n------------------")
    train_loop(training_set,model=model,loss_fn=loss,optimizer=opt,batch_size=1036)
    # with torch.no_grad():
    #     y = pd.DataFrame(Parabola_model(X_tensor),columns=['y'])
    # # print(type(y))
    #     df = pd.concat([X,y],axis=1)
    #     df.plot(kind='scatter',x='X',y='y',s = 1.5)
    #     plt.savefig('C:/Projects/AI-learning/Py-torch proto/training.jpg')

Epoch: 1
------------------
tensor([0])


I0000 00:00:1720770998.187073   14220 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720770998.214495   18073 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720770998.308143   18075 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720770998.402727   18086 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1720770998.724666   14220 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720770998.741284   18090 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720770998.754944   18091 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inferen

Scores:  torch.Size([1, 378])
Output:  torch.Size([1, 126])
tensor([[1.7729e-29, 1.0414e-27, 4.3949e-28, 1.0000e+00, 5.1606e-24, 4.0757e-25,
         3.7603e-24, 3.9330e-24, 1.1052e-23, 2.0463e-24, 1.7618e-23, 1.9552e-23,
         1.2430e-24, 3.5399e-23, 9.1747e-24, 5.6536e-20, 7.9667e-25, 4.0421e-23,
         4.9687e-24, 1.2239e-24, 1.5883e-23, 1.2572e-23, 1.8790e-24, 4.9088e-24,
         7.8649e-24, 2.8168e-24, 9.6320e-24, 1.9232e-24, 5.7314e-23, 9.0051e-24,
         1.4499e-23, 1.0802e-23, 3.2819e-24, 4.1971e-24, 2.1369e-21, 1.7914e-20,
         1.3483e-23, 1.8227e-24, 5.1175e-23, 2.2682e-23, 7.9808e-24, 1.2185e-24,
         2.1568e-23, 1.3552e-23, 3.8321e-23, 1.7558e-24, 5.3581e-24, 2.6682e-24,
         3.6806e-22, 1.4308e-23, 4.9440e-24, 6.1249e-24, 1.8258e-23, 2.5267e-23,
         1.8414e-24, 1.0918e-23, 1.5469e-24, 3.5099e-24, 2.5034e-24, 1.3921e-24,
         1.6909e-23, 1.6476e-24, 1.1888e-23, 9.8987e-24, 1.6628e-23, 1.4418e-23,
         1.7783e-24, 1.7713e-24, 5.8082e-22, 2.57

I0000 00:00:1720770999.008338   14220 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720770999.023102   18107 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720770999.111732   18110 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720770999.227594   18122 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1720770999.532013   14220 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720770999.546634   18124 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720770999.557393   18126 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inferen

Scores:  torch.Size([1, 378])
Output:  torch.Size([1, 126])
tensor([[1.7690e-29, 1.0388e-27, 4.3768e-28, 1.0000e+00, 5.1443e-24, 4.0632e-25,
         3.7484e-24, 3.9205e-24, 1.1018e-23, 2.0397e-24, 1.7565e-23, 1.9491e-23,
         1.2390e-24, 3.5291e-23, 9.1461e-24, 5.6393e-20, 7.9408e-25, 4.0302e-23,
         4.9532e-24, 1.2200e-24, 1.5834e-23, 1.2533e-23, 1.8730e-24, 4.8933e-24,
         7.8408e-24, 2.8078e-24, 9.6026e-24, 1.9170e-24, 5.7142e-23, 8.9769e-24,
         1.4456e-23, 1.0769e-23, 3.2715e-24, 4.1838e-24, 2.1310e-21, 1.7868e-20,
         1.3443e-23, 1.8169e-24, 5.1020e-23, 2.2615e-23, 7.9558e-24, 1.2146e-24,
         2.1504e-23, 1.3511e-23, 3.8205e-23, 1.7501e-24, 5.3414e-24, 2.6596e-24,
         3.6700e-22, 1.4266e-23, 4.9284e-24, 6.1059e-24, 1.8203e-23, 2.5192e-23,
         1.8355e-24, 1.0885e-23, 1.5419e-24, 3.4989e-24, 2.4953e-24, 1.3876e-24,
         1.6858e-23, 1.6423e-24, 1.1852e-23, 9.8693e-24, 1.6576e-23, 1.4375e-23,
         1.7725e-24, 1.7656e-24, 5.7918e-22, 2.56

...

I0000 00:00:1720771065.596373   14220 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720771065.610835   20931 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720771065.700444   20933 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720771065.857583   20942 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1720771066.154417   14220 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720771066.168937   20948 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720771066.185123   20951 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inferen

Scores:  torch.Size([1, 378])
Output:  torch.Size([1, 126])
tensor([[1.7329e-29, 1.0143e-27, 4.2091e-28, 1.0000e+00, 4.9922e-24, 3.9456e-25,
         3.6376e-24, 3.8044e-24, 1.0702e-23, 1.9787e-24, 1.7069e-23, 1.8930e-23,
         1.2012e-24, 3.4286e-23, 8.8793e-24, 5.5060e-20, 7.6991e-25, 3.9196e-23,
         4.8090e-24, 1.1830e-24, 1.5375e-23, 1.2173e-23, 1.8170e-24, 4.7490e-24,
         7.6162e-24, 2.7239e-24, 9.3283e-24, 1.8589e-24, 5.5538e-23, 8.7143e-24,
         1.4052e-23, 1.0465e-23, 3.1745e-24, 4.0599e-24, 2.0761e-21, 1.7432e-20,
         1.3071e-23, 1.7625e-24, 4.9580e-23, 2.1989e-23, 7.7229e-24, 1.1780e-24,
         2.0909e-23, 1.3127e-23, 3.7120e-23, 1.6977e-24, 5.1847e-24, 2.5799e-24,
         3.5705e-22, 1.3873e-23, 4.7830e-24, 5.9283e-24, 1.7692e-23, 2.4488e-23,
         1.7804e-24, 1.0570e-23, 1.4953e-24, 3.3960e-24, 2.4202e-24, 1.3457e-24,
         1.6384e-23, 1.5931e-24, 1.1514e-23, 9.5943e-24, 1.6098e-23, 1.3969e-23,
         1.7192e-24, 1.7124e-24, 5.6382e-22, 2.49

I0000 00:00:1720771066.364450   14220 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720771066.379806   20965 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720771066.467963   20968 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720771066.588520   20980 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1720771066.913783   14220 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720771066.929195   20982 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720771066.939241   20984 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inferen

Scores:  torch.Size([1, 378])
Output:  torch.Size([1, 126])
tensor([[1.7329e-29, 1.0143e-27, 4.2091e-28, 1.0000e+00, 4.9922e-24, 3.9456e-25,
         3.6376e-24, 3.8044e-24, 1.0702e-23, 1.9787e-24, 1.7069e-23, 1.8930e-23,
         1.2012e-24, 3.4286e-23, 8.8793e-24, 5.5060e-20, 7.6991e-25, 3.9196e-23,
         4.8090e-24, 1.1830e-24, 1.5375e-23, 1.2173e-23, 1.8170e-24, 4.7490e-24,
         7.6162e-24, 2.7239e-24, 9.3283e-24, 1.8589e-24, 5.5538e-23, 8.7143e-24,
         1.4052e-23, 1.0465e-23, 3.1745e-24, 4.0599e-24, 2.0761e-21, 1.7432e-20,
         1.3071e-23, 1.7625e-24, 4.9580e-23, 2.1989e-23, 7.7229e-24, 1.1780e-24,
         2.0909e-23, 1.3127e-23, 3.7120e-23, 1.6977e-24, 5.1847e-24, 2.5799e-24,
         3.5705e-22, 1.3873e-23, 4.7830e-24, 5.9283e-24, 1.7692e-23, 2.4488e-23,
         1.7804e-24, 1.0570e-23, 1.4953e-24, 3.3960e-24, 2.4202e-24, 1.3457e-24,
         1.6384e-23, 1.5931e-24, 1.1514e-23, 9.5943e-24, 1.6098e-23, 1.3969e-23,
         1.7192e-24, 1.7124e-24, 5.6382e-22, 2.49

I0000 00:00:1720771067.168571   20999 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: Quadro K600/PCIe/SSE2
W0000 00:00:1720771067.257276   21002 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720771067.416675   21015 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
